## Tushare为金融数据分析提供便捷、快速的接口，与投研和量化策略无缝对接

In [1]:
!pip install tushare

Processing c:\users\huyia\appdata\local\pip\cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472\bs4-0.0.1-cp37-none-any.whl


In [1]:
import tushare as ts
print(ts.__version__)

1.2.54


In [19]:
token = "8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123"
pro = ts.pro_api(token)

In [24]:
import os
#os.listdir()
import pandas as pd
futures_info = pd.read_excel("商品期货信息汇总.xlsx")
futures_info.head(2)

,交易所,品种大类别,品种子类别,Unnamed: 3,品种,收盘价格,开盘价格,最低价,最高价,成交量,...,一手保证金,一手价值,手续费,交易单位（contractunit）/手OR每点价值/乘数,最小变动价位（元/吨）,报价单位,合约在线地址,最后交易日,交易时间,涨跌停
0,上海期货交易所,金属,贵金属,11.0,沪金,278.55,NaN,NaN,NaN,NaN,...,19498.5,278550.0,10元/手,1000.0,0.05,元（人民币）/吨,https://finance.sina.com.cn/futures/quotes/AU1...,最后交易日后连续5个工作日,NaN,NaN
1,大连商品交易所,工业品,NaN,16.0,焦炭,1972.00,NaN,NaN,NaN,NaN,...,15776.0,197200.0,成交金额的万分之0.6,100.0,0.5,元（人民币）/吨,http://www.dce.com.cn/dalianshangpin/yw/fw/ywc...,NaN,NaN,NaN


In [30]:
print("查看Main Contract")
mc= futures_info[['交易所', '主力代号（截至20190804）', '品种']].drop_duplicates().dropna().sort_values('交易所')
mc['exchange'] = 'INE'

## 更改dataframe中满足条件行的特定行的值
SHFE = mc.loc[mc['交易所'] == "上海期货交易所"].index
DCE  = mc.loc[mc['交易所'] == "大连商品交易所"].index
CZCE = mc.loc[mc['交易所'] == "郑州商品交易所"].index
mc.loc[SHFE,'exchange'] = "SHFE"
mc.loc[DCE, 'exchange'] = 'DCE'
mc.loc[CZCE,'exchange'] = 'CZCE'


mc = mc.rename(columns = {'主力代号（截至20190804）':'MainContract'})
mc.MainContract = mc.MainContract.astype(str).str[:4]
mc = mc.iloc[:,1:].reset_index(drop = True).rename(columns = {"品种":'name'})

## SHFE产品名称做相应更改
mc.loc[mc.loc[mc.name =='沪金'].index, 'name'] = '黄金'
mc.loc[mc.loc[mc.name =='沪镍'].index, 'name'] = '镍'
mc.loc[mc.loc[mc.name =='沪锡'].index, 'name'] = '锡'
mc.loc[mc.loc[mc.name =='沪铜'].index, 'name'] = '铜'
mc.loc[mc.loc[mc.name =='燃油'].index, 'name'] = '燃料油'
mc.loc[mc.loc[mc.name =='沪铜'].index, 'name'] = '铜'
mc.loc[mc.loc[mc.name =='螺纹'].index, 'name'] = '螺纹钢'

mc_SHFE = mc.loc[mc.exchange == 'SHFE']
mc_DCE  = mc.loc[mc.exchange == 'DCE']
mc_CZCE = mc.loc[mc.exchange == 'CZCE']
mc_INE  = mc.loc[mc.exchange == 'INE']
mc_SHFE

查看Main Contract


,MainContract,name,exchange
1,1906,黄金,SHFE
2,1909,锌,SHFE
3,1907,镍,SHFE
4,1912,沥青,SHFE
5,1909,纸浆,SHFE
6,1909,铝,SHFE
7,1905,锡,SHFE
8,1909,橡胶,SHFE
9,1910,螺纹钢,SHFE
10,1912,银,SHFE


In [8]:
drop_ind = [1,2,3,10, 14, 15, 19, 20, 21, 22, 23]
left = futures_info.columns[drop_ind]
left

Index(['品种大类别', '品种子类别', 'Unnamed: 3', '品种大类别（归纳定义）', '一手保证金', '一手价值', '报价单位',
       '合约在线地址', '最后交易日', '交易时间', '涨跌停'],
      dtype='object')

## 想办法在tushare里找到接口截取期货的日线数据

In [9]:
futures_info = futures_info.drop(columns = left)
futures_info.head()

,交易所,品种,收盘价格,开盘价格,最低价,最高价,成交量,交易代码,主力代号（截至20190804）,保证金（%）（截至20190804）,手续费,交易单位（contractunit）/手OR每点价值/乘数,最小变动价位（元/吨）
0,上海期货交易所,沪金,278.55,NaN,NaN,NaN,NaN,AU,1906.0,7.0,10元/手,1000.0,0.05
1,大连商品交易所,焦炭,1972.00,NaN,NaN,NaN,NaN,J,1909.0,8.0,成交金额的万分之0.6,100.0,0.5
2,大连商品交易所,铁矿石,514.00,NaN,NaN,NaN,NaN,I,2001.0,8.0,1月、5月、9月合约：成交金额万分之0.6；其它合约：成交金额的万分之0.06,100.0,0.5
3,郑州商品交易所,动力煤（郑煤）,565.00,NaN,NaN,NaN,NaN,ZC,1909.0,6.0,4元/手,100.0,0.2
4,大连商品交易所,焦煤,1193.00,NaN,NaN,NaN,NaN,JM,1909.0,9.0,成交金额的万分之0.6,60.0,0.5


In [10]:
futures_info.columns

Index(['交易所', '品种', '收盘价格', '开盘价格', '最低价', '最高价', '成交量', '交易代码',
       '主力代号（截至20190804）', '保证金（%）（截至20190804）', '手续费',
       '交易单位（contractunit）/手OR每点价值/乘数', '最小变动价位（元/吨）'],
      dtype='object')

## 查看主力合约代号  Main Contract

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
mc= futures_info[['交易所', '主力代号（截至20190804）']].drop_duplicates().dropna().sort_values('交易所')
mc['exchange'] = 'INE'
SHFE = mc.loc[mc['交易所'] == "上海期货交易所"].index
DCE  = mc.loc[mc['交易所'] == "大连商品交易所"].index
CZCE = mc.loc[mc['交易所'] == "郑州商品交易所"].index
mc.loc[SHFE,'exchange'] = "SHFE"
mc.loc[DCE, 'exchange'] = 'DCE'
mc.loc[CZCE,'exchange'] = 'CZCE'
mc = mc.rename(columns = {'主力代号（截至20190804）':'MainContract'})
mc.MainContract = mc.MainContract.astype(str).str[:4]
mc.reset_index(drop = True, inplace = True)
mc.tail()

,交易所,MainContract,exchange
7,大连商品交易所,1909,DCE
8,大连商品交易所,2001,DCE
9,郑州商品交易所,1909,CZCE
10,郑州商品交易所,1910,CZCE
11,郑州商品交易所,2001,CZCE


In [24]:
futures_info['交易所'].unique()

array(['上海期货交易所', '大连商品交易所', '郑州商品交易所', '上海国际能源交易中心', nan], dtype=object)

In [35]:
futures_info['品种'].unique()

array(['沪金', '焦炭', '铁矿石', '动力煤（郑煤）', '焦煤', '银', '螺纹', '热轧卷板', '沪锡', '橡胶',
       '郑醇', '沥青', '豆粕', '菜籽粕（简称“菜粕”）', '豆油', '棕榈（油）', '黄大豆1号', '黄玉米',
       '苹果', '菜籽油（简称“菜油”）（“郑油”）', '白砂糖(简称“白糖”）', '沪铜', '铝', '锌',
       '线型低密度聚乙烯（塑料）', '聚氯乙烯（PVC）', '\t聚丙烯（PP）', '精对苯二甲酸（PTA）', '鸡蛋',
       '棉花', '沪镍', '纸浆', '铅', '原油', '燃油', '玻璃', '乙二醇', '锰硅', '硅铁', '玉米淀粉',
       nan, '淀粉'], dtype=object)

## 交易新闻

In [3]:
ts.get_latest_news()

<urlopen error timed out>


## 指数数据

In [4]:
ts.get_index()

,code,name,change,open,preclose,close,high,low,volume,amount
0,000001,上证指数,0.31,2798.4295,2811.1741,2819.9350,2823.3382,2796.8388,203003773,2224.3260
1,000002,Ａ股指数,0.31,2932.7967,2946.1803,2955.4296,2958.9879,2931.1336,202884288,2223.5897
2,000003,Ｂ股指数,-0.81,215.9379,215.9460,214.1933,215.9379,214.1933,119485,0.7363
3,000008,综合指数,0.15,2596.1436,2609.1392,2613.1415,2617.3912,2595.9677,41952686,441.2754
4,000009,上证380,0.47,4674.3871,4692.9609,4714.8630,4722.4962,4665.4110,42863087,511.3138
5,000010,上证180,0.28,8062.2676,8099.5175,8121.9547,8132.8818,8059.2033,60050616,841.0392
6,000011,基金指数,0.28,6111.6096,6126.6020,6144.0100,6152.9890,6105.0175,89943187,301.6190
7,000012,国债指数,0.02,182.1254,182.1087,182.1488,182.1917,182.1254,597545,6.0387
8,000016,上证50,0.14,2751.6937,2764.4883,2768.2616,2771.4343,2750.2919,23016379,417.7671
9,000017,新综指,0.31,2364.7952,2375.5880,2383.0453,2385.9146,2363.4543,200864208,2127.6969


## 期货合约信息表

In [41]:
print(">>>>>>>>>>>>> 获取期货合约信息表")
print("接口：\tfut_basic\n描述:\t获取期货合约列表数据")
print("可选参数：")
print("\texchange\t|\tCFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心")
print("\tfut_type\t|\t合约类型 (1->普通合约; 2->主力与连续合约 默认取全部)")

>>>>>>>>>>>>> 获取期货合约信息表
接口：	fut_basic
描述:	获取期货合约列表数据
可选参数：
	exchange	|	CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心
	fut_type	|	合约类型 (1->普通合约; 2->主力与连续合约 默认取全部)


In [13]:
pro = ts.pro_api(token)
futures_contract= pro.fut_basic(exchange='INE', fut_type='1', fields='ts_code,symbol,name,list_date,delist_date')

In [14]:
print("\t|合约代码|交易标识|中文简称|上市日期|最后交易日期|")
futures_contract.tail()

	|合约代码|交易标识|中文简称|上市日期|最后交易日期|


,ts_code,symbol,name,list_date,delist_date
50,NR2005.INE,NR2005,20号胶2005,20190812,20200515
51,SC2209.INE,SC2209,原油2209,20190902,20220831
52,SC2011.INE,SC2011,原油2011,20191101,20201030
53,SC2101.INE,SC2101,原油2101,20191223,20201224
54,NR2102.INE,NR2102,20号胶2102,20200218,20210205


## 期货交易日历

In [8]:
print(">>>>>>>>>>>>> 获取期货交易日历")
print("接口：\ttrade_cal\n描述:\t获取各大期货交易所交易日历数据")
print("可选参数：")
print("\texchange\t|\tCFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心")
print("\tstart_date\t|\t开始日期")
print("\tend_date\t|\t结束日期")
print("\tis_open\t\t|\t是否交易【0->休市; 1->交易】")

>>>>>>>>>>>>> 获取期货交易日历
接口：	trade_cal
描述:	获取各大期货交易所交易日历数据
可选参数：
	exchange	|	CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心
	start_date	|	开始日期
	end_date	|	结束日期
	is_open		|	是否交易【0->休市; 1->交易】


In [16]:
futures_trade_cal_DCE = pro.trade_cal(exchange = "DCE")

In [17]:
print("\t|交易所|日历日期|是否交易|")
futures_trade_cal_DCE.tail()

	|交易所|日历日期|是否交易|


,exchange,cal_date,is_open
10163,DCE,20201227,0
10164,DCE,20201228,1
10165,DCE,20201229,1
10166,DCE,20201230,1
10167,DCE,20201231,1


## 期货日线行情

In [21]:
print(">>>>>>>>>>>>> 获取期货日线行情")
print("接口：\tfut_daily\n描述:\t获取期货日线行情数据")
print("可选参数：")
print("\ttrade_date\t|\t交易日期")
print("\tts_code\t\t|\t合约代码")
print("\texchange\t|\tCFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心")
print("\tstart_date\t|\t开始日期")
print("\tend_date\t|\t结束日期")

>>>>>>>>>>>>> 获取期货日线行情
接口：	fut_daily
描述:	获取期货日线行情数据
可选参数：
	trade_date	|	交易日期
	ts_code		|	合约代码
	exchange	|	CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心
	start_date	|	开始日期
	end_date	|	结束日期


In [18]:
futures_daily = pro.fut_daily(ts_code = "CU1811.SHF", start_date = "20180101", end_date = "20191231")
daily_columns = "合约代码|交易日期|昨日收盘价|昨日结算价|开盘价|最高价|最低价|收盘价|结算价|收盘价-昨日结算价|结算价-昨日结算价|成交量|成交金额（万)|持仓量|持仓量变化".split("|")
for c in daily_columns: print("", c, end = "|")
futures_daily.tail()

 合约代码| 交易日期| 昨日收盘价| 昨日结算价| 开盘价| 最高价| 最低价| 收盘价| 结算价| 收盘价-昨日结算价| 结算价-昨日结算价| 成交量| 成交金额（万)| 持仓量| 持仓量变化|

,ts_code,trade_date,pre_close,pre_settle,open,high,low,close,settle,change1,change2,vol,amount,oi,oi_chg
207,CU1811.SHF,20180108,56200.0,56300.0,56010.0,56070.0,55750.0,55910.0,55840.0,-390.0,-460.0,84.0,2345.53,1412.0,48.0
208,CU1811.SHF,20180105,56650.0,56590.0,56600.0,56600.0,56090.0,56200.0,56300.0,-390.0,-290.0,46.0,1295.05,1364.0,20.0
209,CU1811.SHF,20180104,56200.0,56210.0,56110.0,56660.0,56070.0,56650.0,56590.0,440.0,380.0,44.0,1245.12,1344.0,0.0
210,CU1811.SHF,20180103,56610.0,56490.0,56540.0,56580.0,56110.0,56200.0,56210.0,-290.0,-280.0,166.0,4666.23,1344.0,0.0
211,CU1811.SHF,20180102,56820.0,57000.0,56870.0,56870.0,56390.0,56610.0,56490.0,-390.0,-510.0,64.0,1807.72,1344.0,2.0


In [23]:
futures_daily[''] = futures_daily.ts_code.str[2:6]

0      1811
1      1811
2      1811
3      1811
4      1811
       ... 
207    1811
208    1811
209    1811
210    1811
211    1811
Name: ts_code, Length: 212, dtype: object

## 每日成交持仓排名

In [20]:
print(">>>>>>>>>>>>> 获取每日成交持仓排名")
print("接口：\tfut_holding\n描述:\t获取每日成交持仓排名数据")
print("\ttrade_date\t|\t交易日期")
print("\tsymbol\t\t|\t合约或产品代码\t[注：（trade_date/symbol至少输入一个参数）]")
print("\texchange\t|\tCFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心")
print("\tstart_date\t|\t开始日期")
print("\tend_date\t|\t结束日期")

>>>>>>>>>>>>> 获取每日成交持仓排名
接口：	fut_holding
描述:	获取每日成交持仓排名数据
	trade_date	|	交易日期
	symbol		|	合约或产品代码	[注：（trade_date/symbol至少输入一个参数）]
	exchange	|	CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心
	start_date	|	开始日期
	end_date	|	结束日期


In [38]:
futures_hold = pro.fut_holding(trade_date = '20191231', symbol = "B")
print(f"共有{len(futures_hold)}条数据")
hold_columns = "交易日期|Symbol|期货公司会员简称|成交量|成交量变化|持买仓量|持买仓量变化|持卖仓量|持卖仓量变化".split("|")
for c in hold_columns: print("", c, end = " |")
futures_hold.head()

共有36条数据
|交易日期|Symbol|期货公司会员简称|成交量|成交量变化|持买仓量|持买仓量变化|持卖仓量|持卖仓量变化|


,trade_date,symbol,broker,vol,vol_chg,long_hld,long_chg,short_hld,short_chg
0,20191231,B,一德期货,NaN,NaN,NaN,NaN,835.0,328.0
1,20191231,B,上海中期,29080.0,26394.0,2380.0,-14496.0,2315.0,-14386.0
2,20191231,B,东方财富,939.0,-1082.0,NaN,NaN,NaN,NaN
3,20191231,B,东证期货,19121.0,3692.0,1461.0,117.0,379.0,40.0
4,20191231,B,中信建投,NaN,NaN,NaN,NaN,727.0,-304.0


## 仓单日报

In [39]:
print(">>>>>>>>>>>>> 获取仓单日报")
print("接口：\tfut_wsr\n描述:\t获取仓单日报数据，了解各仓库/厂库的仓单变化")
print("\ttrade_date\t|\t交易日期")
print("\texchange\t|\tCFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心")

>>>>>>>>>>>>> 获取仓单日报
接口：	fut_wsr
描述:	获取仓单日报数据，了解各仓库/厂库的仓单变化
	trade_date	|	交易日期
	exchange	|	CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心


In [25]:
futures_wsr = pro.fut_wsr(trade_date ="20200214", exchange = "DCE")#symbol = "ZN")
wsr_columns = "交易日期|产品代码|产品名称|仓库名称|昨日仓单量|今日仓单量|增减量|单位".split("|")
for c in wsr_columns: print("", c, end = " |")
futures_wsr.tail()

 交易日期 | 产品代码 | 产品名称 | 仓库名称 | 昨日仓单量 | 今日仓单量 | 增减量 | 单位 |

,trade_date,symbol,fut_name,warehouse,pre_vol,vol,vol_chg,unit
56,20200214,Y,豆油,天津京粮,3396,3396,0,手
57,20200214,Y,豆油,江海粮油,575,575,0,手
58,20200214,Y,豆油,泰州过船港,487,487,0,手
59,20200214,Y,豆油,秦皇岛金海,1400,1400,0,手
60,20200214,Y,豆油,金光食品,1500,1500,0,手


In [26]:
futures_wsr.unit.unique()

array(['手'], dtype=object)

## 结算参数

In [12]:
print(">>>>>>>>>>>>> 获取结算参数")
print("接口：\tfut_settle\n描述:\t获取每日结算参数数据，包括交易和交割费率等")
print("\ttrade_date\t|\t交易日期")
print("\tts_code\t\t|\t合约代码")
print("\texchange\t|\tCFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心")

>>>>>>>>>>>>> 获取结算参数
接口：	fut_settle
描述:	获取每日结算参数数据，包括交易和交割费率等
	trade_date	|	交易日期
	ts_code		|	合约代码
	exchange	|	CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心


In [21]:
futures_settle = pro.fut_settle(trade_date = "20190308", exchange = "CZCE")
settle_columns = "合约代码|交易日期|结算价|交易手续费率|交易手续费|交割手续费率|买套保交易保证金率|卖套保交易保证金率|买投机保证金率|卖投机保证金率"
settle_columns = settle_columns.split("|")
for c in settle_columns: print("",c, end = " |") 
futures_settle.tail()

 合约代码 | 交易日期 | 结算价 | 交易手续费率 | 交易手续费 | 交割手续费率 | 买套保交易保证金率 | 卖套保交易保证金率 | 买投机保证金率 | 卖投机保证金率 |

,ts_code,trade_date,settle,trading_fee_rate,trading_fee,delivery_fee,b_hedging_margin_rate,s_hedging_margin_rate,long_margin_rate,short_margin_rate
144,ZC1908.ZCE,20190308,597.8,None,4.0,5000.0,None,None,0.08,0.08
145,ZC1909.ZCE,20190308,582.0,None,4.0,5000.0,None,None,0.08,0.08
146,ZC1910.ZCE,20190308,586.8,None,4.0,5000.0,None,None,0.08,0.08
147,ZC1911.ZCE,20190308,581.6,None,0.5,5000.0,None,None,0.08,0.08
148,ZC1912.ZCE,20190308,590.6,None,4.0,5000.0,None,None,0.08,0.08


## 南华期货指数日线行情

In [27]:
print(">>>>>>>>>>>>> 获取南华期货指数日线行情")
print("接口：\tindex_daily\n描述:\t获取南华指数每日行情，指数行情也可以通过通用行情接口获取数据．[https://tushare.pro/document/2?doc_id=109]")
print("\ttrade_date\t|\t交易日期")
print("\tts_code\t\t|\t合约代码")
futures_index_daily = pro.index_daily(ts_code = "CU.NH", start_date='20200101', end_date='20201201')
ind_dai_columns = "TS指数代码，交易日期，收盘点位，开盘点位，最高点位，最低点位，昨日收盘点，涨跌点，涨跌幅，成交量，成交额（千）".split("，")
for i in ind_dai_columns: print("", i, end = " |")
futures_index_daily

>>>>>>>>>>>>> 获取南华期货指数日线行情
接口：	index_daily
描述:	获取南华指数每日行情，指数行情也可以通过通用行情接口获取数据．[https://tushare.pro/document/2?doc_id=109]
	trade_date	|	交易日期
	ts_code		|	合约代码
 TS指数代码 | 交易日期 | 收盘点位 | 开盘点位 | 最高点位 | 最低点位 | 昨日收盘点 | 涨跌点 | 涨跌幅 | 成交量 | 成交额（千） |

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,CU.NH,20200421,3232.313,3265.073,3288.473,3229.193,3301.733,-69.420,-0.0210,259582.0,325857.0
1,CU.NH,20200420,3301.733,3277.553,3325.133,3276.773,3297.833,3.900,0.0012,177276.0,321386.0
2,CU.NH,20200417,3297.833,3260.393,3494.393,3259.613,3241.673,56.160,0.0173,195148.0,317698.0
3,CU.NH,20200416,3241.673,3228.413,3250.253,3222.953,3241.673,0.000,0.0000,128670.0,320661.0
4,CU.NH,20200415,3241.673,3259.613,3261.953,3223.733,3256.493,-14.820,-0.0046,162959.0,334927.0
...,...,...,...,...,...,...,...,...,...,...,...
67,CU.NH,20200108,3827.010,3827.796,3836.257,3809.926,3838.608,-11.598,-0.0030,156825.0,296775.0
68,CU.NH,20200107,3838.608,3813.514,3839.392,3813.514,3819.947,18.661,0.0049,117651.0,296486.0
69,CU.NH,20200106,3819.947,3811.009,3838.146,3802.844,3830.459,-10.512,-0.0027,190723.0,293494.0
70,CU.NH,20200103,3830.459,3857.459,3857.929,3828.104,3865.778,-35.319,-0.0091,209999.0,293581.0


## 期货主力与连续合约

In [22]:
print(">>>>>>>>>>>>> 获取期货主力与连续合约")
print("接口：\tfut_mapping\n描述:\t获取期货主力（或连续）合约与月合约映射数据")
print("\ttrade_date\t|\t交易日期")
print("\tts_code\t\t|\t合约代码")
futures_mapping = pro.fut_mapping(ts_code = "TF.CFX")
mapping_columns = "连续合约代码。起始日期。期货合约代码".split("。")
for i in mapping_columns: print("", i, end = " |")
futures_mapping.tail()

>>>>>>>>>>>>> 获取期货主力与连续合约
接口：	fut_mapping
描述:	获取期货主力（或连续）合约与月合约映射数据
	trade_date	|	交易日期
	ts_code		|	合约代码
 连续合约代码 | 起始日期 | 期货合约代码 |

,ts_code,trade_date,mapping_ts_code
1607,TF.CFX,20130912,TF1312.CFX
1608,TF.CFX,20130911,TF1312.CFX
1609,TF.CFX,20130910,TF1312.CFX
1610,TF.CFX,20130909,TF1312.CFX
1611,TF.CFX,20130906,TF1312.CFX


## 期货主要品种交易周报

In [29]:
print(">>>>>>>>>>>>> 获取期货主要品种交易周报")
print("接口：\tfut_weekly_detail\n描述:\t获取期货交易所主要品种每周交易统计信息，数据从2010年3月开始")
print("\tweek\t\t|\t周期（每年第几周，e.g. 202001 表示2020第1周）")
print("\tprd\t\t|\t期货品种（支持多品种输入，逗号分隔）")
print("\tstart_week\t|\t开始周期")
print("\tend_week\t|\t结束周期")
print("\texchange\t|\tCFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心")
print("\tfields\t\t|\t提取的字段，e.g. fields='prd,name,vol'")
all_columns = "exchange|prd|name|vol|vol_yoy|amount|amount_yoy|cumvol|cumvol_yoy|cumamt|cumamt_yoy|open_interest|interest_wow|mc_close|close_wow|week|week_date".split("|")
print("可选择输出：")
for c in range(len(all_columns)): print(f"{c}:{all_columns[c]}", end = " |")
futures_weekly_detail = pro.fut_weekly_detail(exchange = 'SHFE', start_week = "202001", end_week = "202003", fields = all_columns[:5])
futures_weekly_detail.tail()

>>>>>>>>>>>>> 获取期货主要品种交易周报
接口：	fut_weekly_detail
描述:	获取期货交易所主要品种每周交易统计信息，数据从2010年3月开始
	week		|	周期（每年第几周，e.g. 202001 表示2020第1周）
	prd		|	期货品种（支持多品种输入，逗号分隔）
	start_week	|	开始周期
	end_week	|	结束周期
	exchange	|	CFFEX-中金所 DCE-大商所 CZCE-郑商所 SHFE-上期所 INE-上海国际能源交易中心
	fields		|	提取的字段，e.g. fields='prd,name,vol'
可选择输出：
0:exchange |1:prd |2:name |3:vol |4:vol_yoy |5:amount |6:amount_yoy |7:cumvol |8:cumvol_yoy |9:cumamt |10:cumamt_yoy |11:open_interest |12:interest_wow |13:mc_close |14:close_wow |15:week |16:week_date |

,exchange,prd,name,vol,vol_yoy
43,SHFE,BU,沥青,1097328,0.8157
44,SHFE,RB,螺纹钢,4466780,-6.8969
45,SHFE,HC,热轧卷板,696632,-26.9996
46,SHFE,CU,铜,658216,-14.7655
47,SHFE,WR,线材,38,-99.2265
